<a href="https://colab.research.google.com/github/fernandodeeke/can2025/blob/main/gauss_elim_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1></h1></center>
<center><h1>Análise Numérica</h1></center>
<center><h2>2025/1</h2></center>
<center><h3>Eliminação Gaussiana - Parte 2</h3></center>
<center><h4>Prof. Fernando Deeke Sasse - CCT, UDESC</h4></center>
<center><h3>2025</h3></center>

### 1. Eliminação gaussiana simples - Revisão

Já descrevemos anteriormente as operações elementares de linha, essenciais no processo de eliminação gaussiana simples. Nosso objetivo agora é automatizar o processo definindo uma função em Python capaz de realizar essas etapas de uma só vez. Consideraremos novamente um [sistema linear](https://pt.wikipedia.org/wiki/Sistema_de_equações_lineares) com $n$ equações e $n$ incógnitas para as variáveis $x_1,\ldots, x_n$, da forma:

\begin{align}
a_{11}x_1 + a_{12}x_2 + \cdots + a_{1n}x_n & = b_1 \\\
a_{21}x_1 + a_{22}x_2 + \cdots + a_{2n}x_n & = b_2 \\\
& \vdots \\\
a_{n1}x_1 + a_{m1}x_2 + \cdots + a_{nn}x_n & = b_n
\end{align}

 Em notação matricial o sistema linear é representado na forma
 $A X= B$, sendo

$$
A = \begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\\
a_{21} & a_{22} & \cdots & a_{2n} \\\
\vdots & & & \vdots \\\
a_{n1} & a_{n2} & \cdots & a_{nn}
\end{bmatrix}
 \ \ , \ \
X = \begin{bmatrix}
x_1 \\\ x_2 \\\ \vdots \\\ x_n
\end{bmatrix}
 \ \ , \ \
B = \begin{bmatrix}
b_1 \\\ b_2 \\\ \vdots \\\ b_n
\end{bmatrix}
$$

O chamado método da eliminação gaussiana simples é o método direto mais utilizado para resolver sistemas lineares e é a base para todas as variantes de métodos de eliminação. Ele consiste de duas etapas:

1. Fase de eliminação: realizamos a eliminação dos elementos abaixo da diagonal principal da matriz ampliada $[A|B]$ por meio de transformações elementares sobre matrizes, até obtermos a forma escalonada.
2. Retrosubstituição.

Descreveremos a seguir cada uma destas etapas, já definindo uma função em Python que realizada cada etapa:

### 2. Fase de eliminação
Chamamos *linha pivot* a linha acima da qual todos os elementos abaixo da diagonal principal já foram zerados.
Na matriz ampliada, seja a  linha pivot $L_k$ e uma típica linha $L_i$ abaixo a ser transformada, de modo a zerar o elemento $a_{ik}$. Para isso devemos multiplicar a linha pivot  por $\lambda = a_{ik}/a_{kk}$ e subtrair o resultado de $L_i$, ou seja,

$$
L_i \rightarrow L_i - \lambda L_k\,.
$$

Os elementos da linha $L_i$ são atualizados da seguinte forma:

\begin{align}
&a_{ij} \leftarrow a_{ij} - \lambda a_{kj}\,\qquad j=k, \ldots , n\\
&b_i \leftarrow b_i - \lambda b_k\,.
\end{align}

Como $a_{ik}=0$ por construção, para economizar tempo computacional, ele não precisa ser calculado, de modo podemos tomar $j=k+1, \ldots , n$.

O índice $k$ é  o índice da linha pivot, de modo que $k=1,  \ldots, n-1$ O índice $i$ designa a linha a ser transformada, de modo que $i=k+1,\ldots, n$.

É conveniente lembrar o modo de indexação do Python. O primeiro índice é zero:

In [1]:
import numpy as np

In [2]:
c=np.array([1,2,3,4])
c

array([1, 2, 3, 4])

In [3]:
c[0]

1

In [4]:
c[3]

4

Quando usamos range(i,j), o valor de j não é incluído. Por exemplo,

In [5]:
for j in range(0,3):
    print(c[j])

1
2
3


Notemos que c[3] não foi selecionado.

O elemento c definido acima é formalmente um array 1-dimensional, ou uma matriz com somente uma linha. Seu comprimento é o número de elementos:

In [6]:
len(c)

4

Se tentarmos calcular a transposta de c usando um comando de atribuição, não haverá efeito:  

In [7]:
np.transpose(c)

array([1, 2, 3, 4])

Podemos usar a alternativa:

In [8]:
cT = np.transpose([c])
cT

array([[1],
       [2],
       [3],
       [4]])

Notemos que

In [9]:
len(cT)

4

Alternativamente poderíamos ter definido:

In [10]:
c = np.array([[2,3,4,5]])
c

array([[2, 3, 4, 5]])

de modo que

In [11]:
c.T

array([[2],
       [3],
       [4],
       [5]])

Para uma matriz, o comprimento é número de linhas (listas) que ela contém:

In [12]:
M = np.array([[2,3,4], [3, 4, 5], [2, 3, 7], [3, 4, 1]])
M

array([[2, 3, 4],
       [3, 4, 5],
       [2, 3, 7],
       [3, 4, 1]])

In [13]:
len(M)

4

O algoritmo  de eliminação pode ser realizado em Python da seguinte forma:

In [106]:
import numpy as np

def GaussElimin(A, b):
    # Cria cópias dos arrays para não modificar os originais
    A = A.copy()
    b = b.copy()
    n = A.shape[0]

    # Eliminação para frente
    for k in range(n - 1):
        # Calcula os multiplicadores para as linhas abaixo do pivot
        lam = A[k+1:, k] / A[k, k]
        # Atualiza a submatriz para as linhas k+1 até n-1 e colunas k+1 até n-1
        A[k+1:, k+1:] -= lam[:, None] * A[k, k+1:]
        # Define os elementos abaixo do pivot como zero
        A[k+1:, k] = 0
        # Atualiza o vetor do lado direito
        b[k+1:] -= lam * b[k]

    # Converte b em vetor coluna e concatena com A horizontalmente
    M = np.hstack([A, b.reshape(-1, 1)])
    return M


Testemos a função.

In [111]:
A = np.array([[6.,1.,2.,4.],[5.,11.,-3.,2.],[-3.,4.,3.,5.],[5.,2.,8.,3.]])
A

array([[ 6.,  1.,  2.,  4.],
       [ 5., 11., -3.,  2.],
       [-3.,  4.,  3.,  5.],
       [ 5.,  2.,  8.,  3.]])

In [112]:
B = np.array([2.,-4.,3.,-7.])
print(B)

[ 2. -4.  3. -7.]


 Formemos a matriz aumentada $M$:

In [115]:
M = np.hstack([A, B.reshape(-1, 1)])
M

array([[ 6.,  1.,  2.,  4.,  2.],
       [ 5., 11., -3.,  2., -4.],
       [-3.,  4.,  3.,  5.,  3.],
       [ 5.,  2.,  8.,  3., -7.]])

A expressão B.reshape(-1, 1) transforma o array unidimensional B em um array bidimensional (ou matriz) com uma única coluna. O -1 indica que o NumPy deve calcular automaticamente o número de linhas com base no tamanho original do array.

Realizemos a eliminação gaussiana:

In [114]:
M1=GaussElimin(A,B)
M1

array([[  6.        ,   1.        ,   2.        ,   4.        ,
          2.        ],
       [  0.        ,  10.16666667,  -4.66666667,  -1.33333333,
         -5.66666667],
       [  0.        ,   0.        ,   6.06557377,   7.59016393,
          6.50819672],
       [  0.        ,   0.        ,   0.        ,  -8.77567568,
        -15.38648649]])

### 3.  Fase de retrosubstituição

Começamos a resolver a corresponde última equação do sistema reduzido, ou seja,

$$
x_n=\frac{b_n}{a_{nn}}\,.
$$

Consideremos agora o estágio da retrosubstituição onde $x_n, x_{n-1}, \ldots , x_{k+1} $ já foram computados e devemos computar $x_k$ a partir da equação $k$:

$$
a_{kk}x_k+a_{k k+1} x_{k+1}+ \cdots + a_{k n}x_n=b_k\,.
$$

Resolvendo para $k_k$ obtemos

$$
x_k=\frac{1}{a_{kk}}\left(b_k-a_{k k+1} x_{k+1}- \cdots - a_{k n}x_n\right)= \frac{1}{a_{kk}}\left(b_k-\sum_{j=k+1}^n a_{kj}x_j\right)\,.
$$

Implementemos esta função no Python.

A seguinte função tem como entrada uma matriz de coeficientes na forma escalonada e a matriz coluna da parte homogênea. A saída é solução do correspondente sistema linear.

In [116]:
# Retrosubstituição
def GaussRetro(a, b):
    n = len(b)
    x = np.empty(n, dtype=a.dtype)
    # Calcula o último elemento
    x[-1] = b[-1] / a[-1, -1]
    # Realiza a substituição regressiva a partir do penúltimo elemento
    for k in range(n - 2, -1, -1):
        x[k] = (b[k] - np.dot(a[k, k+1:], x[k+1:])) / a[k, k]
    # Retorna x como vetor coluna
    return x.reshape(-1, 1)

 Usemos o exemplo acima. Inicialmente fatiamos a matriz ampliada. A matriz de coeficientes é dada por

In [117]:
A1 = M1[:,0:4]
A1

array([[ 6.        ,  1.        ,  2.        ,  4.        ],
       [ 0.        , 10.16666667, -4.66666667, -1.33333333],
       [ 0.        ,  0.        ,  6.06557377,  7.59016393],
       [ 0.        ,  0.        ,  0.        , -8.77567568]])

A parte não homogênea é dada por:

In [118]:
B1= M1[:,4]
B1

array([  2.        ,  -5.66666667,   6.50819672, -15.38648649])

Apliquemos agora a função que realiza a retrosubstituição:

In [119]:
X1 = GaussRetro(A1,B1)
X1

array([[-0.32152756],
       [-0.84200801],
       [-1.1210348 ],
       [ 1.75331075]])

Podemos verificar que este resultado é correto. É importante definir novamente o sistema original, uma vez que as matrizes A e B foram modificadas ao longo do processo. Este procedimento é fundamental em Python.

In [120]:
A = np.array([[6.,1.,2.,4.],[5.,11.,-3.,2.],[-3.,4.,3.,5.],[5.,2.,8.,3.]])
B = np.transpose(np.array([[2.,-4.,3.,-7.]]))

In [121]:
A@X1-B

array([[ 0.00000000e+00],
       [-8.88178420e-16],
       [ 8.88178420e-16],
       [-3.55271368e-15]])

Podemos resolver sistemas lineares com métodos diretos usando  comando solve do numpy:

In [26]:
import numpy.linalg as la

In [27]:
x = la.solve(A,B)
print(x)

[[-0.32152756]
 [-0.84200801]
 [-1.1210348 ]
 [ 1.75331075]]


### 4.  Algoritmo de eliminação gaussiana simples completo

Juntemos as duas funções numa só. A atribuição a[i,k]=0 agora pode ser removida pois é irrelevante:

In [122]:
def GaussSolve(A, b):
    """
    Resolve o sistema linear Ax = b usando eliminação gaussiana sem pivoteamento.

    Parâmetros:
      A : matriz n x n (numpy.array)
      b : vetor 1D com n elementos (numpy.array)

    Retorna:
      x : vetor coluna com a solução do sistema.
    """
    # Cria cópias para preservar os dados originais
    A = A.copy()
    b = b.copy()
    n = A.shape[0]

    # Eliminação para frente (transforma A em uma matriz triangular superior)
    for k in range(n - 1):
        # Calcula os multiplicadores para as linhas abaixo do pivot
        lam = A[k+1:, k] / A[k, k]
        # Atualiza as linhas abaixo do pivot utilizando a vetorização
        A[k+1:, k+1:] -= lam[:, None] * A[k, k+1:]
        # Zera os elementos abaixo do pivot
        A[k+1:, k] = 0
        # Atualiza o vetor do lado direito
        b[k+1:] -= lam * b[k]

    # Substituição regressiva (back substitution)
    x = np.empty(n, dtype=A.dtype)
    # Calcula o último elemento da solução
    x[-1] = b[-1] / A[-1, -1]
    # Calcula os demais elementos de trás para frente
    for k in range(n - 2, -1, -1):
        x[k] = (b[k] - np.dot(A[k, k+1:], x[k+1:])) / A[k, k]

    # Retorna a solução como vetor coluna
    return x.reshape(-1, 1)


Testemos o algoritmo completo no sistema linear anterior.

In [123]:
A = np.array([[6.,1.,2.,4.],[5.,11.,-3.,2.],[-3.,4.,3.,5.],[5.,2.,8.,3.]])
B = np.transpose(np.array([2.,-4.,3.,-7.]))

In [124]:
X1=  GaussSolve(A,B)
X1

array([[-0.32152756],
       [-0.84200801],
       [-1.1210348 ],
       [ 1.75331075]])

que é o mesmo resultado obtido anteriormente

### 5. Exemplo com sistemas grandes aleatórios

In [136]:
np.random.seed(43453)
N = 10
A3 = np.random.rand(N,N)
B3 = np.random.rand(N)

In [137]:
X3 = GaussSolve(A3,B3)
X3

array([[-0.78014194],
       [ 1.64411949],
       [-1.13319994],
       [-0.72833755],
       [-2.02054919],
       [ 1.06249775],
       [-0.78987105],
       [ 2.26036594],
       [ 2.07989874],
       [-0.4584725 ]])

Verifiquemos a resposta, calculando o resíduo:

In [140]:
R=A3@X3-B3.reshape(-1, 1)
R

array([[-4.44089210e-16],
       [-3.33066907e-16],
       [-5.10702591e-15],
       [-7.32747196e-15],
       [-2.22044605e-15],
       [ 2.10942375e-15],
       [-4.44089210e-16],
       [-3.21964677e-15],
       [-1.99840144e-15],
       [ 7.77156117e-16]])

Se o sistema é muito grande é difícil inspecionar todas as componentes. Em geral avaliamos o resíduo calculando uma norma do vetor correspondente. A norma mais usada é a norma infinito, é definida como sendo a maior magnitude entre as componentes:  

$$
|\mathbf{u}|_{\infty}=\max_i\{{|u_i|},i=1,\ldots,n\}.
$$

No nosso caso,

In [141]:
import numpy.linalg as la

In [142]:
NR = la.norm(R, np.inf)
NR

7.327471962526033e-15

Testemos o desempenho do programa que escrevemos com sistemas grandes.

In [143]:
np.random.seed(43453)
N = 300
A4 = np.random.rand(N,N)
B4 = np.random.rand(N)

In [144]:
X4 = GaussSolve(A4,B4)

Calculemos o resíduo:

In [145]:
NR = la.norm(A4@X4-B4.reshape(-1, 1),np.inf)
NR

6.478706460200101e-12

Examinemos o tempo de CPU necessário para resolver este sistema:

In [146]:
np.random.seed(43453)
N = 300
A4 = np.random.rand(N,N)
B4 = np.random.rand(N)

In [147]:
%%timeit
GaussSolve(A4,B4)

36.5 ms ± 3.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


O tempo de CPU necessário usando a função solve do numpy é dado por:

In [148]:
np.random.seed(43453)
N = 300
A4 = np.random.rand(N,N)
B4 = np.random.rand(N)

In [149]:
%%timeit
la.solve(A,B)

6.52 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [150]:
la.solve(A,B)

array([-0.32152756, -0.84200801, -1.1210348 ,  1.75331075])

Como esperado, o solver é muito mais rápido do que nosso algoritmo pedagógico.

### 6. Exercícios

1. Resolva passo a passo um sistema linear $4 \times 4$ aleatório (use os 4 primeiros dígitos do seu cpf para o seed), descrevendo cada passo do algoritmo GaussElimin e GaussRetro. Verifique a resposta calculando o resíduo.
<p>
2. Resolva um sistema aleatório $500 \times 500$ usando o algoritmo GaussSolve (não mostre o resultado). Verifique a correção do resultado calculando a norma do resíduo e compare o tempo de CPU necessário. Repita o procedimento usando o solver do numpy.

3. O **número de condição** de uma matriz $ A $, denotado por $ \kappa(A) $, mede a sensibilidade da solução de um sistema linear $ A \mathbf{x} = \mathbf{b} $ a tais pequenas perturbações na matriz $ A $ ou no vetor $ \mathbf{b} $. Ele é definido como:
$$
\kappa(A) = \|A\| \cdot \|A^{-1}\|,
$$
onde $ \|A\| $ é a norma da matriz (normalmente a norma 2). Um número de condição alto indica que o sistema é mal condicionado, ou seja, pequenas mudanças nos dados podem causar grandes variações na solução.
Um exemplo de mal condicionada é a matriz de Hilbert $ H_n $ de ordem $ n $,  definida por
$$
H_{ij} = \frac{1}{i + j + 1}, \quad i, j = 0, 1, \dots, n.
$$

Podemos defini-la no Python da seguinte forma:

In [ ]:
def hilb(n):
    return np.array([[1/(i+j+1) for i in range(n)] for j in range(n)])

Por exemplo,

In [ ]:
hilb(4)

array([[1.        , 0.5       , 0.33333333, 0.25      ],
       [0.5       , 0.33333333, 0.25      , 0.2       ],
       [0.33333333, 0.25      , 0.2       , 0.16666667],
       [0.25      , 0.2       , 0.16666667, 0.14285714]])

(i) Resolva o sistema linear $HX=B$, sendo $H$ a matriz de Hilbert $H(30)$, $30 \times 30$ e B uma matriz coluna com todos elementos de valor 1. Use o algoritmo de eliminação gaussiana desenvolvido anteriormente para resolver o sistema.  Calcule o resíduo em ambos os casos.

(ii) Compare o número de condição de $H$ com aquele de uma matriz aleatória $30 \times 30$.

(iii) Mostre que uma pequena perturbação em uma componente de $H(30)$ faz com que a solução do sistema definido em (i) muda drasticamente.